In [18]:
"""
Experiments with 
Explainable Verbal Deception Detection using TransformersLoukas Ilias1, Felix Soldner2,3, and Bennett Kleinberg3,4
"""
import openai
import os

openai.api_key = os.environ["OPENAI_API_KEY"]  # source the ~/.zshrc file

# constants, until you change them :-)
new_line = '\n'
test_samples_count = 10
nb_samples_of_each_class = 8
MODEL = "gpt-3.5-turbo"
# MODEL = "gpt-4"


In [23]:
import pandas as pd
df = pd.read_csv ('sign_events_data_statements.csv')
# simple EDA
# print(df)
# print(df.columns)
print(f'shape: {df.shape}')  # should be 1640 x 6


shape: (1640, 6)


In [24]:
def filter_by_class(df, category):
   return df[df['outcome_class']== category]

truth_df = filter_by_class(df, 't')
# print(truth_df)
print(f'truth df shape: {truth_df.shape}')  # should be 1640 x 6

# replace with a more expressive word, truthful
truth_df['outcome_class'] = df['outcome_class'].replace('t','truthful')
print(truth_df)

deceit_df = filter_by_class(df, 'd')
# print(deceit_df)
print(f'deceit df shape: {deceit_df.shape}')  # should be 1640 x 6

# replace with a more expressive word, deceitful
deceit_df['outcome_class'] = df['outcome_class'].replace('d','deceitful')
print(deceit_df)

# pick random non-repeating rows
def randon_non_repeating(df, quantity):
    import random
    rand_df = pd.DataFrame()
    random_list = random.sample(range(df.shape[0]), quantity)
    print("non-repeating random numbers are:")
    return df.iloc[random_list], random_list

random_truth_df, truth_indices_list = randon_non_repeating(truth_df, nb_samples_of_each_class)
print(f'random truth list:\n, {random_truth_df}')
print(f'truth indices:" {truth_indices_list}')

random_deceit_df, deceit_indices_list = randon_non_repeating(deceit_df, nb_samples_of_each_class)
print(f'random deceit list:\n, {random_deceit_df}')
deceit_indices_list = [x + truth_df.shape[0] for x in deceit_indices_list] # do this to exclude from poriginal list
print(f'deceit indices: {deceit_indices_list}')

random_truth_deceit_df = pd.concat([random_truth_df, random_deceit_df])
few_shot_list = truth_indices_list + deceit_indices_list
print(f'truth + deceit indices" {few_shot_list}')


truth df shape: (783, 6)
                                             signevent   
0                                  My brothers wedding  \
1                   Going to collect 2 new pet rabbits   
2    Getting dinner with my friend Shan and my boyf...   
3    mountain bike ride with my boyfriend and daughter   
4    I will be going to the cat cafe in Glasgow on ...   
..                                                 ...   
778                          Go to NJ to visit cousins   
779  I am going to visit my mum and gran who I've n...   
780                                           Swimming   
781                    Going for coffee with the girls   
782                    Going to the cinema with Yasmin   

                                                    q1   
0    My little brother is getting married next Satu...  \
1    I will be driving for 80-90 minutes across Lon...   
2    We were planning to get dinner somewhere near ...   
3    We are going to cannock chase with the mo

/tmp/ipykernel_5149/2229232140.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truth_df['outcome_class'] = df['outcome_class'].replace('t','truthful')
/tmp/ipykernel_5149/2229232140.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deceit_df['outcome_class'] = df['outcome_class'].replace('d','deceitful')


In [20]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [29]:
intro = f"""
You are a detective tasked to judge whether the response is truthful or deceitful.
You'll be presented with three pieces of information:
(1) The title of an activity.
(2) Response to question #1: “Please describe your activity. Be as specific as possible.”
(3) Response to question #2: “What information can you give us to reassure us that you are telling the truth”
Here are a few examples:
"""

def construct_scenario(row):
    event_header = 'Activity:'
    activity_description_header = 'Question #1: Please describe your activity. Be as specific as possible.'

    activity_reassurance_header = 'Question #2: What information can you give us to reassure us that you are telling the truth'

    event = event_header + new_line + row['signevent'] + new_line
    q1 = activity_description_header + new_line + row['q1'] + new_line
    q2 = activity_reassurance_header + new_line + row['q2'] + new_line
    return event + q1 + q2

def construct_outcome(row):
    outcome = "Your judgement:"
    return outcome + new_line + row['outcome_class'] + new_line * 2
    

def construct_few_shot_prompt(few_shot_df, infer_row):
    # constructed as a list
    prompt = []
    prompt.append(intro)

    for _, row in few_shot_df.iterrows():
        prompt.append(construct_scenario(row))
        prompt.append(construct_outcome(row))
    
    prompt.append(construct_scenario(infer_row))
    prompt.append(construct_outcome(infer_row)) # has to have a black outcome to be filled by the llm
    return prompt

def create_test_indices(df, total, exclude_list):
    import random
    rand_list = []
    count = 0
    while count < total:
        rand_row = random.randrange(df.shape[0])
        if rand_row not in exclude_list:
            rand_list.append(rand_row)
            count += 1
    return rand_list


test_indices = create_test_indices(df, test_samples_count, few_shot_list)  # exclude the ones in the few shot list
print(f'test indices: {test_indices}')

y_ground_truth = []  # for computing F1-score
y_predicted = []

for index in test_indices:
    infer_row = df.loc[index]  
    # print(f'Inferring the `class_outcome` for:\n{infer_row}')
    ground_truth = 'truthful' if infer_row['outcome_class'] == 't' else 'deceitful'
    # mask the `outcome_class` field since you want to predict it
    infer_row['outcome_class'] = ''

    # print(f'Original\n:{df.loc[index]}')
    # print(f'infer row\n: {infer_row}')

    prompt = construct_few_shot_prompt(random_truth_deceit_df, infer_row)
    prompt = ''.join(prompt)
    print(f'{prompt}')    
    response = get_completion(
        prompt=prompt,
        model=MODEL,
    )    
        
    print(f'GROUND TRUTH: {ground_truth}, RESPONSE: {response} - {"wrong" if ground_truth != response else "correct"}')
    y_ground_truth.append(ground_truth)
    y_predicted.append(response)




test indices: [734, 1548, 1288, 442, 871, 348, 1471, 14, 693, 485]

You are a detective tasked to judge whether the response is truthful or deceitful.
You'll be presented with three pieces of information:
(1) The title of an activity.
(2) Response to question #1: “Please describe your activity. Be as specific as possible.”
(3) Response to question #2: “What information can you give us to reassure us that you are telling the truth”
Here are a few examples:
Activity:
Visiting some friends and going out for lunch together to try a new Jerk chicken place.
Question #1: Please describe your activity. Be as specific as possible.
We will all meet at S's house as it's closest to the Jerk place. It will be me, S and A there with our children. We are not sure if we will order the Jerk for the children as it will be too spicy so we will see if they have other dishes for them or order from a different place. We will be eating take away so could order from more than one place. S recommend the Jerk p

/tmp/ipykernel_5149/3474920888.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infer_row['outcome_class'] = ''


GROUND TRUTH: truthful, RESPONSE: truthful - correct

You are a detective tasked to judge whether the response is truthful or deceitful.
You'll be presented with three pieces of information:
(1) The title of an activity.
(2) Response to question #1: “Please describe your activity. Be as specific as possible.”
(3) Response to question #2: “What information can you give us to reassure us that you are telling the truth”
Here are a few examples:
Activity:
Visiting some friends and going out for lunch together to try a new Jerk chicken place.
Question #1: Please describe your activity. Be as specific as possible.
We will all meet at S's house as it's closest to the Jerk place. It will be me, S and A there with our children. We are not sure if we will order the Jerk for the children as it will be too spicy so we will see if they have other dishes for them or order from a different place. We will be eating take away so could order from more than one place. S recommend the Jerk place but neith

### Compute metrics

In [25]:
from sklearn.metrics import f1_score
print('Weighted F1-score:', f1_score(y_ground_truth, y_predicted, average='weighted'))

Weighted F1-score: 0.4
